In [ ]:
###Made by Khalil Droubi
###To process Iolite baseline-subtracted NP-II files for BB

To Do:

-Test out Error minimization

-Determine if filtering is the correct way to remove negative measurements from dataset.

In [ ]:
import pandas as pd
import os
import re
import copy
import numpy as np
from scipy import stats
from scipy.stats import linregress

#Graphing stuff
from matplotlib import pyplot as plt
%matplotlib inline

# %pip install seaborn
# import seaborn as sns

#%pip install PyPDF2
from PyPDF2 import PdfFileMerger, PdfFileReader

#%pip install pdfkit
import pdfkit

#pd.set_option("display.precision", 8)

In [ ]:
#Functions for calculation and EXCEL export
def read_np2_timeseries(excel_file):
    ''' Excel input file is your baseline corrected time series export from Iolite for the NP-II.'''
    df = pd.read_excel(excel_file, sheet_name = None)
    keys = df.keys()
    header_row = 0
    new_dict = {}
    for key in keys:
        if '.' in key: #Kind of hard-coded right now, so if names get weird may need to change
            
            df_test = df[key]

            df_test.columns = df_test.iloc[header_row]
            df_test = df_test.drop(header_row)
            df_test = df_test.reset_index(drop=True)
            
#             test1_new = df_test[['Absolute Time',
#              'Elapsed Time',
#                  'm238_CPS',
#                  'm232_CPS',
#                  'm208_CPS',
#                  'm207_CPS',
#                  'm206_CPS',
#                  'm204_CPS',
#                  'm202_CPS',]]
            
            new_string = key.split('time')[0].rstrip()
            new_dict[new_string] = df_test #test1_new
    return new_dict

def calc_CPS(np2_dict):
    columns = ['Absolute Time',
     'Elapsed Time',
     'm238_CPS',
     'm232_CPS',
     'm208_CPS',
     'm207_CPS',
     'm206_CPS',
     'm204_CPS',
     'm202_CPS']

    new_col = ['Absolute Time',
     'Elapsed Time',
     '238_CPS',
     '232_CPS',
     '208_CPS',
     '207_CPS',
     '206_CPS',
     '204_CPS',
     '202_CPS']

    cut_col = ['238_CPS',
     '232_CPS',
     '208_CPS',
     '207_CPS',
     '206_CPS',
     '204_CPS',
     '202_CPS']

    calc_dict = {}
    for key in np2_dict:
        #print(key)
        test_df1 = np2_dict[key]

        for col in columns:
                test_df2 = test_df1.apply(lambda x: x * 62500000 if 'CPS' in x.name else x)
                test_df2 = test_df2[['Absolute Time',
             'Elapsed Time',
                 'm238_CPS',
                 'm232_CPS',
                 'm208_CPS',
                 'm207_CPS',
                 'm206_CPS',
                 'm204_CPS',
                 'm202_CPS',]]
        test_df2.columns = new_col
        test_df2 = test_df2[cut_col]
        result = pd.concat([test_df1, test_df2], axis=1)
        
         #Calculating OPZ
        result['OPZ_238'] = result.apply(lambda x: x['m238'] - x['m238_CPS'], axis=1)
        result['OPZ_232'] = result.apply(lambda x: x['m232'] - x['m232_CPS'], axis=1)
        result['OPZ_208'] = result.apply(lambda x: x['m208'] - x['m208_CPS'], axis=1)
        result['OPZ_207'] = result.apply(lambda x: x['m207'] - x['m207_CPS'], axis=1)
        result['OPZ_206'] = result.apply(lambda x: x['m206'] - x['m206_CPS'], axis=1)
        result['OPZ_204'] = result.apply(lambda x: x['m204'] - x['m204_CPS'], axis=1)
        result['OPZ_202'] = result.apply(lambda x: x['m202'] - x['m202_CPS'], axis=1)
        
        #Calculating Ratios
        result['206/238'] = result.apply(lambda x: x['206_CPS']/x['238_CPS'], axis=1)
        result['208/232'] = result.apply(lambda x: x['208_CPS']/x['232_CPS'], axis=1)
        result['207/206'] = result.apply(lambda x: x['207_CPS']/x['206_CPS'], axis=1)
        result['208/206'] = result.apply(lambda x: x['208_CPS']/x['206_CPS'], axis=1)
        result['206/204'] = result.apply(lambda x: x['206_CPS']/x['204_CPS'], axis=1)
        result['208/204'] = result.apply(lambda x: x['208_CPS']/x['204_CPS'], axis=1)
        result['207/204'] = result.apply(lambda x: x['207_CPS']/x['204_CPS'], axis=1)
        
        calc_dict[key] = result
    
    return calc_dict

def ranked_minimization(sheet, ratio, reject_percentage = 20):

    mytest = tester[sheet].copy(deep=True)

    df_mean_before = mytest[ratio].mean()
    df_1std_before = mytest[ratio].std()
    df_count_before = mytest[ratio].count()
    df_2se_perc_before = (2 * mytest[ratio].sem()) / df_mean_before * 100

    dif_mean = ratio + '_dif_from_mean'
    dif_1SD = ratio + '_dif_from_1SD'
    mytest[dif_mean] = mytest.apply(lambda x: abs(x[ratio] - df_mean_before), axis=1)
    mytest[dif_1SD] = mytest.apply(lambda x: x[dif_mean] - df_1std_before, axis=1)


    mytest2 = mytest.sort_values(by = dif_1SD, ascending = False)
    #mytest2.head()

    ratios_to_reject = int(mytest[ratio].count() * reject_percentage / 100)
    #print(ratios_to_reject)

    after_rejection = mytest2[ratios_to_reject:]

    df_mean_after = after_rejection[ratio].mean()
    df_1std_after = after_rejection[ratio].std()
    df_count_after = after_rejection[ratio].count()
    df_2se_perc_after = (2 * after_rejection[ratio].sem()) / df_mean_after * 100

    # print(df_mean_after)
    # print(df_1std_after)
    # print(df_2se_perc_after)

    results_dict = {}
    
    results_dict['avg_before'] = df_mean_before
    results_dict['1sd_before'] = df_1std_before
    results_dict['2se%_before'] = df_2se_perc_before
    results_dict['avg_after'] = df_mean_after
    results_dict['1sd_after'] = df_1std_after
    results_dict['2se%_after'] = df_2se_perc_after
    
    return results_dict

def statistics_NP2(calc_dict):
    calc_list = ['238_CPS', '232_CPS',
           '208_CPS', '207_CPS', '206_CPS', '204_CPS', '202_CPS', '206/238',
           '208/232', '207/206', '208/206', '206/204','208/204','207/204'  ]
    mega_dict = {}

    for sheet in calc_dict:
        tester = calc_dict[sheet]
        stats_dict = {}
        for col in tester:

            if col in calc_list:
                #print(col)
                if '/' in col:
                    key = col + '_before rejection'
                else:
                    key = col + '_mean'
                df_mean = tester[col].mean()
                stats_dict[key] = df_mean
                df_precision = (2 * tester[col].sem()) / df_mean * 100
                stats_dict[col + '_se%'] = df_precision
            if 'OPZ' in col:
                 stats_dict[col + '_mean'] = tester[col].mean()
                
        stats_dict['Time (s)'] = tester['Elapsed Time'].max()
        
        #new_string = sheet.replace('time series data', '')
        new_string = sheet.split('time')[0].rstrip()
        mega_dict[new_string] = stats_dict

    df_1 = pd.DataFrame(mega_dict)
    df_flip = pd.DataFrame.transpose(df_1)
    return df_flip

def statistics_ranktest(calc_dict):
    calc_list = ['238_CPS', '232_CPS',
           '208_CPS', '207_CPS', '206_CPS', '204_CPS', '202_CPS', '206/238',
           '208/232', '207/206', '208/206', '206/204','208/204','207/204'  ]
    mega_dict = {}

    for sheet in calc_dict:
        tester = calc_dict[sheet]
        stats_dict = {}
        for col in tester:

            if col in calc_list:
                #print(col)
                if '/' in col:
                    key_bf = col + '_before rejection'
                    key_af = col + '_after rejection'
                    key_bf_se = col + '_before rejection 2se%'
                    key_af_se = col + '_after rejection 2se%'
                    
                    ranked_dict = ranked_minimization(sheet, col)
                    stats_dict[key_bf] = ranked_dict['avg_before']
                    stats_dict[key_bf_se] = ranked_dict['2se%_before']
                    stats_dict[key_af] = ranked_dict['avg_after']
                    stats_dict[key_af_se] = ranked_dict['2se%_after']
                else:
                    key = col + '_mean'
                    df_mean = tester[col].mean()
                    stats_dict[key] = df_mean
                    df_precision = (2 * tester[col].sem()) / df_mean * 100
                    stats_dict[col + '_se%'] = df_precision
            if 'OPZ' in col:
                 stats_dict[col + '_mean'] = tester[col].mean()
                
        stats_dict['Time (s)'] = tester['Elapsed Time'].max()
        
        #new_string = sheet.replace('time series data', '')
        new_string = sheet.split('time')[0].rstrip()
        mega_dict[new_string] = stats_dict

    df_1 = pd.DataFrame(mega_dict)
    df_flip = pd.DataFrame.transpose(df_1)
    return df_flip

def files_ranked_toEXCEL(calc_dict, excel_name):
    stats = statistics_ranktest(calc_dict)
    with pd.ExcelWriter(excel_name) as writer:
        for sheet in calc_dict:
            calc_dict[sheet].to_excel(writer, sheet_name = sheet, index = False)
        
        stats.to_excel(writer, sheet_name = 'Statistics', index = True)
    new_filename = str(excel_name.split('.')[0]) + '_statistics.xlsx'
    with pd.ExcelWriter(new_filename) as writer:
        stats.to_excel(writer, sheet_name = 'Statistics', index = True)

def files_process_toEXCEL(calc_dict, excel_name):
    with pd.ExcelWriter(excel_name) as writer:
        for sheet in calc_dict:
            calc_dict[sheet].to_excel(writer, sheet_name = sheet, index = False)
        
        statistics_NP2(calc_dict).to_excel(writer, sheet_name = 'Statistics', index = True)

def file_process_combine(filename):
    calc_dict = calc_CPS(read_np2_timeseries(filename))
    new_filename = str(filename.split('.')[0]) + '_processed.xlsx'
    files_process_toEXCEL(calc_dict, new_filename)
    

In [ ]:
#Functons for graphing and report generation
def U_Pb_plots(calc_dict, sample, choice = True):
    key_list = ['238_CPS', '232_CPS',
       '208_CPS', '207_CPS', '206_CPS', '204_CPS', '202_CPS', '206/238',
       '208/232', '207/206', '208/206', '206/204']
    
    zet = calc_dict[sample]
    new_string = sample.split('time')[0].rstrip()
    y_list = []
    for key in key_list:
        y_list.append(zet[key])
    
    x = zet['Elapsed Time']
    
    fig, axs = plt.subplots(4, 3, sharex = True, figsize = (12, 12))
    fig.suptitle(new_string, fontsize=24)
    
    ax_list = [
        axs[0, 0], 
        axs[0, 1],   
        axs[0, 2], 
        axs[1, 0], 
        axs[1, 1],
        axs[1, 2],
        axs[2, 0], 
        axs[2, 1], 
        axs[2, 2],
        axs[3, 0], 
        axs[3, 1], 
        axs[3, 2]   
        ]

    axs[0, 0].plot(x, y_list[0])
    axs[0, 1].plot(x, y_list[1])
    axs[0, 2].plot(x, y_list[2])
    axs[1, 0].plot(x, y_list[3])
    axs[1, 1].plot(x, y_list[4])
    axs[1, 2].plot(x, y_list[5])
    axs[2, 0].plot(x, y_list[6])
    axs[2, 1].plot(x, y_list[7])
    axs[2, 2].plot(x, y_list[8])
    axs[3, 0].plot(x, y_list[9])
    axs[3, 0].set(xlabel = 'Time (s)')
    axs[3, 1].plot(x, y_list[10])
    axs[3, 1].set(xlabel = 'Time (s)')
    axs[3, 2].plot(x, y_list[11])
    axs[3, 2].set(xlabel = 'Time (s)')
    for idx in range(len(ax_list)):
        ax_list[idx].ticklabel_format(axis='y', style='sci', scilimits=(0,0))
        ax_list[idx].set_title(key_list[idx])
        y_mean = [np.mean(y_list[idx])]*len(x)
        # Plot the average line
        mean_line = ax_list[idx].plot(x,y_mean, label='Mean', linestyle='--', color = "black")
        # Make a legend
        legend = ax_list[idx].legend(loc='upper right')
    
    MYDIR = ("Figures")
    CHECK_FOLDER = os.path.isdir(MYDIR)

    # If folder doesn't exist, then create it.
    if not CHECK_FOLDER:
        os.makedirs(MYDIR)
        #print("created folder : ", MYDIR)
    
    #new_string = sample.replace('time series data', '').rstrip()
    
    filename = os.path.join(MYDIR, new_string + '.pdf')
    plt.savefig(filename)
    print('Plot for ', new_string, " is complete.")
    
    if choice == False:
        plt.close()
    #else:
        #plt.close()
        

def U_Pb_report(calc_dict, intro_filename, intro = False, output_name = 'U-Pb_output.pdf'):
    MYDIR = ("Figures")
    mergedObject = PdfFileMerger()
    
    if intro:
        mergedObject.append(PdfFileReader(intro_filename, 'rb'))
        print(f'Succesfully incorporated {intro_filename} into PDF.')
    pd.set_option('precision', 2)
    stats = statistics_NP2(calc_dict)
    stat_dict = {}
    stat_dict['stat1'] = stats.iloc[:, 14:]
    stat_dict['stat2'] = stats.iloc[:, :8]
    stat_dict['stat3'] = stats.iloc[:, 8:14]
    html_list = []

    for key in stat_dict:
        name = key + ".pdf"
        stats_html = stat_dict[key].to_html()
        pdfkit.from_string(stats_html, name)
        mergedObject.append(PdfFileReader(name, 'rb'))

    file_list = []
    keys = calc_dict.keys()

    for key in keys:
        #print(key)
        U_Pb_plots(calc_dict, key, False)
        new_string = key.split('time')[0].rstrip()
        filename = os.path.join(MYDIR, new_string + '.pdf')

        mergedObject.append(PdfFileReader(filename, 'rb'))

    if '.pdf' in output_name:
        pass
    else:
        output_name = output_name + '.pdf'
    
    #output_name = "U-Pb_output.pdf"  
    mergedObject.write(output_name)

    print(f'PDF file named: {output_name} is complete.')        
        


In [ ]:
#Currently deprecated. Need to update to reflect calc_CPS()

def calc_CPS2(np2_dict):
    ''' Eliminates negative values in 238, 232, 208, 207, 206, and 204'''
    columns = ['Absolute Time',
     'Elapsed Time',
     'm238_CPS',
     'm232_CPS',
     'm208_CPS',
     'm207_CPS',
     'm206_CPS',
     'm204_CPS',
     'm202_CPS']

    new_col = ['Absolute Time',
     'Elapsed Time',
     '238_CPS',
     '232_CPS',
     '208_CPS',
     '207_CPS',
     '206_CPS',
     '204_CPS',
     '202_CPS']

    cut_col = ['238_CPS',
     '232_CPS',
     '208_CPS',
     '207_CPS',
     '206_CPS',
     '204_CPS',
     '202_CPS']
    key_list = ['m238_CPS',
     'm232_CPS',
     'm208_CPS',
     'm207_CPS',
     'm206_CPS',
     'm204_CPS']

    calc_dict = {}
    for key in np2_dict:
        #print(key)
        test_df1 = np2_dict[key]
        

        test_df_new = test_df1
        for item in key_list:
            filter_mass = test_df_new[item] > 0
            test_df_new = test_df_new[filter_mass]
            #print(test_df_new.shape)
        test_df1 = test_df_new
        
        for col in columns:
                test_df2 = test_df1.apply(lambda x: x * 62500000 if 'CPS' in x.name else x)
        test_df2.columns = new_col
        test_df2 = test_df2[cut_col]
        result = pd.concat([test_df1, test_df2], axis=1)
        
       
        
        
        #Calculating ratios
        result['206/238'] = result.apply(lambda x: x['206_CPS']/x['238_CPS'], axis=1)
        result['208/232'] = result.apply(lambda x: x['208_CPS']/x['232_CPS'], axis=1)
        result['207/206'] = result.apply(lambda x: x['207_CPS']/x['206_CPS'], axis=1)
        result['208/206'] = result.apply(lambda x: x['208_CPS']/x['206_CPS'], axis=1)
        result['206/204'] = result.apply(lambda x: x['206_CPS']/x['204_CPS'], axis=1)
        
        calc_dict[key] = result
    
    return calc_dict

def file_process_combine2(filename):
    calc_dict = calc_CPS2(read_np2_timeseries(filename))
    new_filename = str(filename.split('.')[0]) + '_processed.xlsx'
    files_process_toEXCEL(calc_dict, new_filename)

In [ ]:
filename = '06May_ttnSS2_NP2_baseline_corrected.xlsx'
test_df = read_np2_timeseries(filename)

In [ ]:
test_df.keys()
test_df['SRM NIST 610 1.1']

In [ ]:
tester = calc_CPS(test_df)
tester.keys()

In [ ]:
# excel_name = '06May_ttnSS2_NP2_processed.xlsx'
# files_ranked_toEXCEL(tester, excel_name)

In [ ]:
tester['SRM NIST 610 1.1']

In [ ]:
statistics_NP2(tester).keys()

In [ ]:
statistics_ranktest(tester)

In [ ]:
#INPUT
reject_percentage = 20

sheet = 'SRM NIST 610 1.1'
ratio = '208/232'
#END INPUT
    


ranked_minimization(sheet, ratio, reject_percentage)

In [ ]:
#filename = '0416_glass_SS_NPII_baseline.xlsx'
#calc_dict = calc_CPS(read_np2_timeseries(filename))

In [ ]:
# filename = '06May_ttnSS2_NP2_baseline_corrected.xlsx'
# SS_dict = calc_CPS(read_np2_timeseries(filename))
# excel_name = str(filename.split('.')[0]) + '_processed.xlsx'

# files_process_toEXCEL(SS_dict, excel_name)

# U_Pb_report(SS_dict, 'SS2_6May.pdf', True, 'Splitstream_06May2021_results')

In [ ]:
# filename = '0416_ttn_SS_NP2_data1.xlsx'
# ttn_dict = calc_CPS(read_np2_timeseries(filename))

# U_Pb_report(ttn_dict, 'titaniteTE.pdf',True, 'Titanite_splitstream_results')

In [ ]:
#U_Pb_report(calc_dict, 'glassTE.pdf',True, 'Glass_splitstream_results')